In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import gseapy as gp
from tqdm import tqdm


In [3]:
go_file = pd.read_csv('data/go_terms.csv', index_col=0)
go_file.loc[go_file['GO']=='GO:0061740']

,GO,Genes,Gene_Count,Term_Description
7918,GO:0061740,HSPA8 LAMP2 CLU,3,protein targeting to lysosome involved in chap...


In [25]:
# create my own GO library from the 11.15.2023 version of the GO database
go_file = pd.read_csv('data/go_terms.csv', index_col=0)
# go_file.head()
# Transform the data to GMT format
gmt_data = go_file.apply(lambda x: f"{x['Term_Description']}({x['GO']})\t\t" + "\t".join(x['Genes'].split()), axis=1)

# Create a string from the transformed data
gmt_content = "\t\n".join(gmt_data)
# Specify the path for the new GMT file
gmt_file_path = 'data/GO_BP_20231115.gmt'

# Write the data to a GMT file
with open(gmt_file_path, 'w') as file:
    file.write(gmt_content)




In [ ]:
gmt_file_path = 'data/GO_BP_20231115.gmt'
gmt_go = gp.read_gmt(gmt_file_path)
gmt_go


In [107]:
df = pd.read_csv('data/GO_term_analysis/100_selected_go_contaminated.csv', index_col=0)

gene_cols = [col for col in df.columns if col.endswith('Genes')]
print(gene_cols)

# print(df.shape)
geneSeparator = ' '

['Genes', '50perc_contaminated_Genes', '100perc_contaminated_Genes']


In [108]:
enrich_res_df = df.copy()
for col in gene_cols:
    df[f'{col} enriched term id'] = None
    df[f'{col} enriched term description'] = None
    df[f'{col} adj p-value'] = 1
    df[f'{col} overlap over enriched'] = 0
    df[f'{col} overlaping genes'] = None
    print(col)
    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        # print(i)
        genes = row[col].split(geneSeparator)
        # print(genes)
        enr = gp.enrich(gene_list=genes,
                        gene_sets='./data/GO_BP_20231115.gmt',
                        outdir=None,
                        verbose=False)
        enr_res = enr.results
        # print(enr_res)
        enr_res = enr_res.sort_values(by='Adjusted P-value', ascending=True, ignore_index=True)
        # print(enr_res)  
        top_term = enr_res['Term'][0]
        
        term_description = top_term.split('(')[0]
        term_id = top_term.split('(')[1].replace(')', '')
        adj_pval = enr_res['Adjusted P-value'][0]
        overlap = enr_res['Overlap'][0]
        overlap_genes = ','.join(enr_res['Genes'][0].split(';'))
            # print(term_description,term_id, adj_pval, overlap, overlap_genes)
        enrich_res_df.loc[i, f'{col} enriched term id'] = term_id
        enrich_res_df.loc[i, f'{col} enriched term description'] = term_description
        enrich_res_df.loc[i, f'{col} adj p-value'] = adj_pval
        enrich_res_df.loc[i, f'{col} overlap over enriched'] = overlap
        enrich_res_df.loc[i, f'{col} overlaping genes'] = overlap_genes

    enrich_res_df.to_csv(f'data/GO_term_analysis/100_GO_terms_enricher_res.tsv', sep='\t', index=True)
            

Genes


100%|██████████| 100/100 [10:41<00:00,  6.41s/it]


50perc_contaminated_Genes


100%|██████████| 100/100 [10:20<00:00,  6.20s/it]


100perc_contaminated_Genes


100%|██████████| 100/100 [09:31<00:00,  5.72s/it]


In [4]:
cutoff = 0.05
# check if the enriched term id equals to the index 
print(sum(enrich_res_df['Genes enriched term id'] == enrich_res_df.index), 'of the enriched term ids are equal to the queried GO id')
# check number of Nones in the enriched term id column
print(sum(enrich_res_df['Genes adj p-value']>cutoff), 'of the real GO term failed to return enriched term')
print(sum(enrich_res_df['50perc_contaminated_Genes adj p-value']>cutoff), 'of the contaminated GO term failed to return enriched term')
print(sum(enrich_res_df['100perc_contaminated_Genes adj p-value']>cutoff), 'of the random GO term failed to return enriched term')

92 of the enriched term ids are equal to the queried GO id
0 of the real GO term failed to return enriched term
3 of the contaminated GO term failed to return enriched term
85 of the random GO term failed to return enriched term


In [20]:
# check for the 9 umatched GO terms, are htey the go terms with the same set of genes but different names
def fill_duplicate_enrich(res_df, go_table_path = 'data/go_terms.csv'):
    go_table = pd.read_csv(go_table_path, index_col=0)
    go_table.set_index('GO', inplace=True)
    
    unmatched = res_df.loc[res_df['Genes enriched term id'] != res_df.index, :]
    for i, row in unmatched.iterrows():
        enriched_GO_id = row['Genes enriched term id']
        query_genes = row['Genes'].split(' ')
        if enriched_GO_id not in go_table.index.values:
            print(f'enriched term {enriched_GO_id} not in the go table!')
        
        else:
            enriched_genes = go_table.loc[enriched_GO_id, ['Genes']].values[0].split(' ')
        if set(query_genes) == set(enriched_genes):
            print('same set of genes')
            res_df.loc[i, 'Genes enriched term id'] = i
            res_df.loc[i, 'Genes enriched term description'] = go_table.loc[enriched_GO_id, 'Term_Description']
            continue
        else:
            print('different set of genes')
            print('queried term', i, ' ', row['Term_Description'])
            print('enriched term', row['Genes enriched term id'], ' ', row['Genes enriched term description'])
            print('queried genes', row['Genes'])
            print('enriched genes', go_table.loc[enriched_GO_id, 'Genes'])
            print('-------------------')
            
    return res_df

enrich_res_df = pd.read_csv('data/GO_term_analysis/100_GO_terms_enricher_res.tsv', sep='\t', index_col=0)
enrich_res_df = fill_duplicate_enrich(enrich_res_df)

same set of genes
same set of genes
same set of genes
same set of genes
same set of genes
same set of genes
same set of genes
same set of genes


In [22]:
cutoff = 0.05
# check if the enriched term id equals to the index 
print(sum(enrich_res_df['Genes enriched term id'] == enrich_res_df.index), 'of the enriched term ids are equal to the queried GO id')
# check number of Nones in the enriched term id column
print(sum(enrich_res_df['Genes adj p-value']>cutoff), 'of the real GO term failed to return enriched term')
print(sum(enrich_res_df['50perc_contaminated_Genes adj p-value']>cutoff), 'of the contaminated GO term failed to return enriched term')
print(sum(enrich_res_df['100perc_contaminated_Genes adj p-value']>cutoff), 'of the random GO term failed to return enriched term')

enrich_res_df.to_csv(f'data/GO_term_analysis/100_GO_terms_enricher_res.tsv', sep='\t', index=True)

100 of the enriched term ids are equal to the queried GO id
0 of the real GO term failed to return enriched term
3 of the contaminated GO term failed to return enriched term
85 of the random GO term failed to return enriched term


In [36]:
cutoff = 0.05
col_keep = [col for col in enrich_res_df.columns if '100perc_contaminated' in col]
false_pos_enrichr = enrich_res_df.loc[enrich_res_df['100perc_contaminated_Genes adj p-value']<cutoff, col_keep ]
false_pos_idx = false_pos_enrichr.index.values
false_pos_enrichr

,100perc_contaminated_Genes,100perc_contaminated_Genes enriched term id,100perc_contaminated_Genes enriched term description,100perc_contaminated_Genes adj p-value,100perc_contaminated_Genes overlap over enriched,100perc_contaminated_Genes overlaping genes
GO,,,,,,
GO:0061740,VEZT TERF1 TTC3,GO:0032214,negative regulation of telomere maintenance vi...,0.013348,1/2,TERF1
GO:0141085,TBC1D24 NIPSNAP2 IL1RL2 EGF PSMG2 GCSAML LIN7A...,GO:0045199,maintenance of epithelial cell apical/basal po...,0.002576,3/11,"LHX2,WDR1,LIN7A"
GO:0008292,CDH11 SLC22A8 RHOBTB2,GO:0003189,aortic valve formation,0.046875,1/2,CDH11
GO:0048840,TSTD1 SCGN BORA MARVELD3 ERO1A STX5 TSPAN4 EDE...,GO:0035966,response to topologically incorrect protein,0.019772,3/157,"ERO1A,EDEM1,DNAJB5"
GO:0006105,AKAP12 LBR PPP3CB CHRNA10 SNAP23 ATP6V0D2 PCGF1,GO:0007268,chemical synaptic transmission,0.025191,4/739,"PPP3CB,SNAP23,AKAP12,CHRNA10"
GO:0000296,SPACA5 CAMSAP3 KNL1,GO:0010256,endomembrane system organization,0.012853,3/594,"CAMSAP3,KNL1,SPACA5"
GO:2000347,ATP6V1G1 ZNF821 OR8J3 COX16 NCF4 ATP6V0A2 H1-4...,GO:0036295,cellular response to increased oxygen levels,0.007050,2/12,"ATP6V1G1,ATP6V0A2"
GO:1903651,MRPS16 NDUFAF6 KARS1 CD300C SLC35F4,GO:0045575,basophil activation,0.046691,1/3,KARS1
GO:1902966,PLXNA3 PTH2R OR2S2 TRIM17 MTHFR RFTN2 BHMT2 CA...,GO:0006555,methionine metabolic process,0.002844,2/16,"BHMT2,MTHFR"
